## Fixed Data:

In [2]:
pip install fredapi
import json
import requests
import pandas as pd
from fredapi import Fred

API_KEY = '1vGb0FOsg2hEIGrEhpueLCcWd1Y'
FRED_API_KEY = 'b13a905b98b5dfc1f3910d6a0d875ae3'

rootUrl = 'https://api.glassnode.com'

symbolList = ['BTC', 'ETH', 'LTC', 'AAVE', 'ABT', 'AMPL', 'ANT', 'ARMOR', 'BADGER', 'BAL', 'BAND', 'BAT', 'BIX', 'BNT', 'BOND', 'BRD', 'BUSD', 'BZRX', 'CELR', 'CHSB', 'CND', 'COMP', 'CREAM', 'CRO', 'CRV', 'CVC', 'CVP', 'DAI', 'DDX', 'DENT', 'DGX', 'DHT', 'DMG', 'DODO', 'DOUGH', 'DRGN', 'ELF', 'ENG', 'ENJ', 'EURS', 'FET', 'FTT', 'FUN', 'GNO', 'GUSD', 'HEGIC', 'HOT', 'HPT', 'HT', 'HUSD', 'INDEX', 'KCS', 'LAMB', 'LBA', 'LDO', 'LEO', 'LINK', 'LOOM', 'LRC', 'MANA', 'MATIC',
              'MCB', 'MCO', 'MFT', 'MIR', 'MKR', 'MLN', 'MTA', 'MTL', 'MX', 'NDX', 'NEXO', 'NFTX', 'NMR', 'Nsure', 'OCEAN', 'OKB', 'OMG', 'PAY', 'PERP', 'PICKLE', 'PNK', 'PNT', 'POLY', 'POWR', 'PPT', 'QASH', 'QKC', 'QNT', 'RDN', 'REN', 'REP', 'RLC', 'ROOK', 'RPL', 'RSR', 'SAI', 'SAN', 'SNT', 'SNX', 'STAKE', 'STORJ', 'sUSD', 'SUSHI', 'TEL', 'TOP', 'UBT', 'UMA', 'UNI', 'USDC', 'USDK', 'USDP', 'USDT', 'UTK', 'VERI', 'WaBi', 'WAX', 'WBTC', 'WETH', 'wNXM', 'WTC', 'YAM', 'YFI', 'ZRX']

symbolListShort = ['BTC', 'ETH', 'LTC']

endPoints = {
    'Price': '/v1/metrics/market/price_usd_close',
    'MarketCap': '/v1/metrics/market/marketcap_usd',
    'TransactionCount': '/v1/metrics/transactions/count',
    'ActiveAddress': '/v1/metrics/addresses/active_count',
}

### <font color='red'> It seems the risk-free rate is daily rate. is it in percentage or in unit 1? need to adjust to weekly rate and select Monday

In [134]:
fred = Fred(api_key=FRED_API_KEY)
Risk_free_rate = fred.get_series('DGS1MO').to_frame().reset_index().rename(columns={'index':'Date', 0:'RiskFree'})

## Get Multiple Attributes for Multiple Coins:

In [82]:
### Define a Method to Pull Data According to Input Prameter(coin, feature)

def PullData(coin, feature):
    print(f"Pulling {coin}'s {feature}")

    # Assemble full request url and get responses:
    reqUrl = rootUrl + endPoints[feature]
    res = requests.get(reqUrl,
                       params={'a': coin, 'api_key': API_KEY, 'i':'1w'})
    data_raw = pd.read_json(res.text, convert_dates=['t'])

    # Rename column "v" according to features
    data = data_raw.rename(columns={"v" : f"{feature}", 't':'Date'})

    # Return a dataframe
    return data

In [136]:
# Effect:

PullData('BTC', 'Price').head()

Pulling BTC's Price


,Date,Price
0,2010-07-12,0.08584
1,2010-07-19,0.05050
2,2010-07-26,0.06110
3,2010-08-02,0.06090
4,2010-08-09,0.06529


In [98]:
### Define a Method to Get Features(Listed in "endPoints"(Constant)) of Coins(Listed in "coinList"(Method Prameter)) 

def PullStart(coinList):

    # Initialization
    from functools import reduce

    colName = ["Date"]
    for key, value in endPoints.items():
        colName.append(key) 
    fullFrame = pd.DataFrame(columns = colName.append("Asset"))

    # Data Pull
    for i in coinList:
        print(f"Start Pulling {i}'s data:")

        # Preparation for merging
        frameName = f"{i}_Frames"
        locals()[frameName] = []

        # Get data
        for j in list(endPoints.keys()):
            locals()[frameName].append(PullData(i, j))

        # Merge 2-column dataframes to a single dataframe
        dfName = f"{i}_All"
        # Outer Merge to Ensure Not Missing Any Values.
        locals()[dfName] = reduce(lambda left, right: pd.merge(left,right, on='Date', how='outer'), locals()[frameName]) 
        locals()[dfName]['Asset'] = i
        locals()[dfName] = locals()[dfName]
        
        # Left Merge Fred to ensure all the records in api stored.
        locals()[dfName] = locals()[dfName].merge(Risk_free_rate, how = 'left').sort_values(by=['Date'])

        # Merge All Coins into a single Dataframe
        fullFrame = fullFrame.append(locals()[dfName], ignore_index=True)

        print(f"{i} data successfully pulled\n")
    print('\nAll Frames Merged')
    print('\nALL DATA SUCCESSFULLY PULLED!')

    return fullFrame

In [99]:
# Effect:

df = PullStart(symbolListShort)
df

Start Pulling BTC's data:
Pulling BTC's Price
Pulling BTC's MarketCap
Pulling BTC's TransactionCount
Pulling BTC's ActiveAddress
BTC data successfully pulled

Start Pulling ETH's data:
Pulling ETH's Price
Pulling ETH's MarketCap
Pulling ETH's TransactionCount
Pulling ETH's ActiveAddress
ETH data successfully pulled

Start Pulling LTC's data:
Pulling LTC's Price
Pulling LTC's MarketCap
Pulling LTC's TransactionCount
Pulling LTC's ActiveAddress
LTC data successfully pulled


All Frames Merged

ALL DATA SUCCESSFULLY PULLED!


,Date,Price,MarketCap,TransactionCount,ActiveAddress,Asset,RiskFree
0,2008-12-29,NaN,NaN,NaN,0,BTC,0.04
1,2009-01-05,NaN,NaN,NaN,0,BTC,0.05
2,2009-01-12,NaN,NaN,19.0,34,BTC,0.04
3,2009-01-19,NaN,NaN,9.0,63,BTC,NaN
4,2009-01-26,NaN,NaN,5.0,61,BTC,0.02
...,...,...,...,...,...,...,...
1592,2022-03-21,129.241438,9.039261e+09,748866.0,1514571,LTC,0.22
1593,2022-03-28,128.926292,9.023978e+09,807733.0,1566202,LTC,0.19
1594,2022-04-04,111.343007,7.798857e+09,841901.0,1576883,LTC,0.18
1595,2022-04-11,109.018201,7.641529e+09,832321.0,1652475,LTC,0.22


## <font color='red'> Imputation for NA values -- to be resolved

## Calculate Returns

In [133]:
### Define a Method to Add Return Column()
def AddReturn(sample):
    fullFrame = pd.DataFrame(columns = list(sample.columns).append('Return'))
    for index in sample.groupby('Asset').count().index.values.tolist():
        syntax = f"Asset == '{index}'"
        sagment = sample.query(syntax)
        # Only Select 53 Weeks for Test
        temp = sagment[-53:-1].reset_index().drop('index', axis=1)
        sagment = sagment[-52:].reset_index().drop('index', axis=1)
        sagment['Return'] = (sagment['Price'] - temp['Price'])/temp['Price']
        fullFrame = fullFrame.append(sagment, ignore_index=True)
    return fullFrame

In [132]:
# Effect:

AddReturn(df)

,Date,Price,MarketCap,TransactionCount,ActiveAddress,Asset,RiskFree,Return
0,2021-04-26,56613.202648,1.058532e+12,1861992.0,5778871,BTC,0.02,0.154432
1,2021-05-03,58218.566077,1.088991e+12,2038316.0,6622607,BTC,0.02,0.028357
2,2021-05-10,46406.331544,8.683518e+11,1908851.0,6062366,BTC,0.02,-0.202895
3,2021-05-17,34888.678313,6.530094e+11,1607054.0,4960283,BTC,0.00,-0.248191
4,2021-05-24,35639.981455,6.672647e+11,1508758.0,4981835,BTC,0.01,0.021534
...,...,...,...,...,...,...,...,...
151,2022-03-21,129.241438,9.039261e+09,748866.0,1514571,LTC,0.22,0.125519
152,2022-03-28,128.926292,9.023978e+09,807733.0,1566202,LTC,0.19,-0.002438
153,2022-04-04,111.343007,7.798857e+09,841901.0,1576883,LTC,0.18,-0.136382
154,2022-04-11,109.018201,7.641529e+09,832321.0,1652475,LTC,0.22,-0.020880


## MOM Factor

### Get LargeCap Sample(maket cap > 10000)

In [112]:
# Get LargeCoin List

largeList = []
print('Finding All Coins with More Than 1m Cap......')
for i in symbolList:
    if pd.read_json(requests.get(rootUrl + endPoints['MarketCap'], params={'a': i, 'api_key': API_KEY, 'i':'1w'}).text, convert_dates=['t']).iloc[-1,1] > 1000000:
        largeList.append(i)
print("Done!")

Finding All Coins with More Than 1m Cap......
Done!


In [116]:
# Get largeCapSample_raw Data(Without Return)

largeCapSample_raw = PullStart(largeList)

Start Pulling BTC's data:
Pulling BTC's Price
Pulling BTC's MarketCap
Pulling BTC's TransactionCount
Pulling BTC's ActiveAddress
BTC data successfully pulled

Start Pulling ETH's data:
Pulling ETH's Price
Pulling ETH's MarketCap
Pulling ETH's TransactionCount
Pulling ETH's ActiveAddress
ETH data successfully pulled

Start Pulling LTC's data:
Pulling LTC's Price
Pulling LTC's MarketCap
Pulling LTC's TransactionCount
Pulling LTC's ActiveAddress
LTC data successfully pulled

Start Pulling AAVE's data:
Pulling AAVE's Price
Pulling AAVE's MarketCap
Pulling AAVE's TransactionCount
Pulling AAVE's ActiveAddress
AAVE data successfully pulled

Start Pulling ABT's data:
Pulling ABT's Price
Pulling ABT's MarketCap
Pulling ABT's TransactionCount
Pulling ABT's ActiveAddress
ABT data successfully pulled

Start Pulling AMPL's data:
Pulling AMPL's Price
Pulling AMPL's MarketCap
Pulling AMPL's TransactionCount
Pulling AMPL's ActiveAddress
AMPL data successfully pulled

Start Pulling ANT's data:
Pulling 

In [119]:
# Add Return to largeCapSample_raw

largeCapSample = AddReturn(largeCapSample_raw)
largeCapSample

,Date,Price,MarketCap,TransactionCount,ActiveAddress,Asset,RiskFree,Return
0,2021-04-26,483.109487,6.033093e+09,0.0,4294.0,AAVE,0.02,0.393008
1,2021-05-03,459.310213,5.735887e+09,0.0,9738.0,AAVE,0.02,-0.049263
2,2021-05-10,525.296787,6.648981e+09,0.0,10036.0,AAVE,0.02,0.143665
3,2021-05-17,296.307038,3.779533e+09,0.0,12152.0,AAVE,0.00,-0.435925
4,2021-05-24,324.282384,4.141262e+09,0.0,11055.0,AAVE,0.01,0.094413
...,...,...,...,...,...,...,...,...
6235,2022-03-21,31.941824,5.356529e+07,0.0,184.0,wNXM,0.22,0.112223
6236,2022-03-28,34.426101,5.773133e+07,0.0,90.0,wNXM,0.19,0.077775
6237,2022-04-04,33.251072,5.576085e+07,0.0,195.0,wNXM,0.18,-0.034132
6238,2022-04-11,27.854533,4.671105e+07,0.0,255.0,wNXM,0.22,-0.162297


### Reshape Dataframe

In [120]:
# Get Return DataFrame

returnAll = largeCapSample.loc[:,['Date', 'Return', 'Asset']].pivot(index="Date", columns=["Asset"], values='Return').reset_index().rename_axis(None, axis=1).sort_values('Date')
returnAll

,Date,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,BAND,BAT,...,VERI,WAX,WBTC,WTC,WaBi,YAM,YFI,ZRX,sUSD,wNXM
0,2021-04-26,0.393008,0.148764,-0.073592,0.219156,0.028645,0.450839,0.282477,0.307409,0.229047,...,-0.047122,NaN,0.149613,0.326124,0.541436,0.153995,0.214523,0.394776,-0.003986,0.191180
1,2021-05-03,-0.049263,0.022970,0.112456,0.069679,0.080898,-0.024938,0.079109,0.001570,0.126182,...,0.406858,NaN,0.032826,0.141704,0.070760,-0.145964,0.102180,0.046813,0.010270,0.348608
2,2021-05-10,0.143665,-0.171471,-0.281857,-0.244447,-0.323333,-0.256346,-0.182583,-0.204940,-0.194722,...,0.062182,NaN,-0.204564,-0.164128,-0.319218,0.064864,0.234226,-0.195655,-0.007335,-0.150428
3,2021-05-17,-0.435925,-0.482439,-0.357192,-0.560776,-0.456955,-0.597872,-0.523110,-0.514725,-0.504094,...,-0.447383,NaN,-0.245743,-0.593304,-0.551696,-0.382593,-0.499353,-0.486727,-0.019204,-0.410748
4,2021-05-24,0.094413,0.379361,0.464830,0.228098,0.103387,0.117416,0.069808,0.085198,0.244939,...,0.188722,NaN,0.019047,0.204790,0.212288,-0.279703,0.272157,0.161636,0.005509,0.241871
5,2021-05-31,0.126079,-0.039411,0.076882,0.345243,-0.040499,0.302944,0.054685,0.090265,0.082609,...,0.046409,NaN,0.005559,-0.025069,0.090076,0.100535,0.028074,0.128756,0.000937,0.052810
6,2021-06-07,-0.133376,-0.041374,0.027304,-0.209913,-0.215166,-0.145077,-0.180341,-0.136070,-0.151720,...,0.746187,NaN,0.082746,-0.060055,-0.093995,-0.044256,-0.105978,-0.156050,0.000112,-0.179796
7,2021-06-14,-0.135142,-0.054374,-0.010388,-0.064211,-0.148949,-0.075485,-0.184541,-0.109692,-0.071613,...,-0.325650,NaN,-0.080900,-0.050993,0.062245,0.145024,-0.061840,-0.112552,-0.002604,-0.074501
8,2021-06-21,-0.260413,-0.225371,-0.034417,-0.151840,-0.203397,-0.335214,-0.156782,-0.170017,-0.097705,...,0.064259,NaN,-0.029888,-0.286174,-0.298918,-0.294125,-0.165332,-0.196463,0.011801,-0.105652
9,2021-06-28,0.368870,0.343686,0.262778,0.076755,-0.047041,0.195057,0.537898,0.209763,0.117593,...,0.024084,NaN,0.020096,0.217856,0.039831,0.033109,0.108094,0.134017,-0.009815,0.134316


### Calculate Momentum Matrix(return 1-week.)

In [121]:
# Need to be Changed into 2-week Cumulative Return

momMatrix = returnAll.copy()
momMatrix['Date'] = momMatrix['Date'].shift(-1)
momMatrix = momMatrix.drop(momMatrix.tail(1).index)
momMatrix.head()

,Date,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,BAND,BAT,...,VERI,WAX,WBTC,WTC,WaBi,YAM,YFI,ZRX,sUSD,wNXM
0,2021-05-03,0.393008,0.148764,-0.073592,0.219156,0.028645,0.450839,0.282477,0.307409,0.229047,...,-0.047122,NaN,0.149613,0.326124,0.541436,0.153995,0.214523,0.394776,-0.003986,0.191180
1,2021-05-10,-0.049263,0.022970,0.112456,0.069679,0.080898,-0.024938,0.079109,0.001570,0.126182,...,0.406858,NaN,0.032826,0.141704,0.070760,-0.145964,0.102180,0.046813,0.010270,0.348608
2,2021-05-17,0.143665,-0.171471,-0.281857,-0.244447,-0.323333,-0.256346,-0.182583,-0.204940,-0.194722,...,0.062182,NaN,-0.204564,-0.164128,-0.319218,0.064864,0.234226,-0.195655,-0.007335,-0.150428
3,2021-05-24,-0.435925,-0.482439,-0.357192,-0.560776,-0.456955,-0.597872,-0.523110,-0.514725,-0.504094,...,-0.447383,NaN,-0.245743,-0.593304,-0.551696,-0.382593,-0.499353,-0.486727,-0.019204,-0.410748
4,2021-05-31,0.094413,0.379361,0.464830,0.228098,0.103387,0.117416,0.069808,0.085198,0.244939,...,0.188722,NaN,0.019047,0.204790,0.212288,-0.279703,0.272157,0.161636,0.005509,0.241871


### Calculate MOM Percentile(30%, 70%)

In [122]:
# Only elect Assets(BTC, ETH...) columns
col = []
for colName in momMatrix.columns:
     if colName not in ['Date','MOM30p','MOM70p']:
         col.append(colName)
momMatrix[col].head()

,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,BAND,BAT,BIX,...,VERI,WAX,WBTC,WTC,WaBi,YAM,YFI,ZRX,sUSD,wNXM
0,0.393008,0.148764,-0.073592,0.219156,0.028645,0.450839,0.282477,0.307409,0.229047,0.596650,...,-0.047122,NaN,0.149613,0.326124,0.541436,0.153995,0.214523,0.394776,-0.003986,0.191180
1,-0.049263,0.022970,0.112456,0.069679,0.080898,-0.024938,0.079109,0.001570,0.126182,-0.168886,...,0.406858,NaN,0.032826,0.141704,0.070760,-0.145964,0.102180,0.046813,0.010270,0.348608
2,0.143665,-0.171471,-0.281857,-0.244447,-0.323333,-0.256346,-0.182583,-0.204940,-0.194722,-0.131835,...,0.062182,NaN,-0.204564,-0.164128,-0.319218,0.064864,0.234226,-0.195655,-0.007335,-0.150428
3,-0.435925,-0.482439,-0.357192,-0.560776,-0.456955,-0.597872,-0.523110,-0.514725,-0.504094,-0.509419,...,-0.447383,NaN,-0.245743,-0.593304,-0.551696,-0.382593,-0.499353,-0.486727,-0.019204,-0.410748
4,0.094413,0.379361,0.464830,0.228098,0.103387,0.117416,0.069808,0.085198,0.244939,0.010949,...,0.188722,NaN,0.019047,0.204790,0.212288,-0.279703,0.272157,0.161636,0.005509,0.241871


In [123]:
# 30% pencentile
momMatrix[col].quantile(.3, axis=1).head()

0    0.154039
1   -0.020051
2   -0.211393
3   -0.517193
4    0.070218
Name: 0.3, dtype: float64

In [124]:
momMatrix['MOM30p']=momMatrix[col].quantile(.3, axis=1)
momMatrix['MOM70p']=momMatrix[col].quantile(.7, axis=1)
momMatrix.tail()

,Date,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,BAND,BAT,...,WBTC,WTC,WaBi,YAM,YFI,ZRX,sUSD,wNXM,MOM30p,MOM70p
46,2022-03-21,0.313923,0.021279,0.128621,0.088104,0.338948,0.136350,0.111796,0.087379,0.215996,...,0.088652,0.155870,0.149697,0.067380,0.105221,0.159266,-0.003560,0.137464,0.055774,0.135960
47,2022-03-28,0.107949,0.086414,0.058932,0.198973,0.519558,0.622892,0.188726,0.265803,0.102234,...,0.134572,0.155009,0.230889,0.180514,0.115208,0.319245,0.003350,0.112223,0.086211,0.190274
48,2022-04-04,0.437923,0.022216,-0.054472,0.063506,0.184376,-0.040351,0.156682,0.189886,0.046700,...,-0.005221,0.072453,0.099797,0.018065,0.133097,0.254326,0.002601,0.077775,0.009694,0.098713
49,2022-04-11,-0.264748,-0.076450,0.019980,-0.222887,-0.118711,-0.159381,0.060292,-0.263378,-0.211052,...,-0.091150,-0.119842,-0.099201,-0.141089,-0.197513,-0.171324,-0.004978,-0.034132,-0.171571,-0.084241
50,2022-04-18,-0.061670,0.195216,-0.005664,-0.089618,-0.141436,-0.142879,-0.156324,-0.103066,-0.030664,...,-0.060917,-0.128957,-0.137774,-0.058160,-0.073668,-0.089801,-0.006177,-0.162297,-0.103619,-0.034644


In [125]:
# reshape for later use
momPt = momMatrix[['MOM30p','MOM70p']]
momMatrix = momMatrix.drop(['MOM30p','MOM70p'], axis=1)

### Calculate Momentum portfolio returns

In [126]:
# delete the first 1 row of returnAll to match the momMatrix date
returnAll = returnAll.drop(returnAll.head(1).index).reset_index().drop('index', axis=1)
returnAll.head()

,Date,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,BAND,BAT,...,VERI,WAX,WBTC,WTC,WaBi,YAM,YFI,ZRX,sUSD,wNXM
0,2021-05-03,-0.049263,0.022970,0.112456,0.069679,0.080898,-0.024938,0.079109,0.001570,0.126182,...,0.406858,NaN,0.032826,0.141704,0.070760,-0.145964,0.102180,0.046813,0.010270,0.348608
1,2021-05-10,0.143665,-0.171471,-0.281857,-0.244447,-0.323333,-0.256346,-0.182583,-0.204940,-0.194722,...,0.062182,NaN,-0.204564,-0.164128,-0.319218,0.064864,0.234226,-0.195655,-0.007335,-0.150428
2,2021-05-17,-0.435925,-0.482439,-0.357192,-0.560776,-0.456955,-0.597872,-0.523110,-0.514725,-0.504094,...,-0.447383,NaN,-0.245743,-0.593304,-0.551696,-0.382593,-0.499353,-0.486727,-0.019204,-0.410748
3,2021-05-24,0.094413,0.379361,0.464830,0.228098,0.103387,0.117416,0.069808,0.085198,0.244939,...,0.188722,NaN,0.019047,0.204790,0.212288,-0.279703,0.272157,0.161636,0.005509,0.241871
4,2021-05-31,0.126079,-0.039411,0.076882,0.345243,-0.040499,0.302944,0.054685,0.090265,0.082609,...,0.046409,NaN,0.005559,-0.025069,0.090076,0.100535,0.028074,0.128756,0.000937,0.052810


In [127]:
# initialize df for low, medium and high MOM
lowMom = pd.DataFrame(columns = ['Date', 'LowMomentum'])
lowMom['Date'] = momMatrix['Date']

medMom = pd.DataFrame(columns = ['Date', 'MediumMomentum'])
medMom['Date'] = momMatrix['Date']

highMom = pd.DataFrame(columns = ['Date', 'HighMomentum'])
highMom['Date'] = momMatrix['Date']

In [128]:
# iterate through whole dataframe to get low, medium and high MOM
lowList = []
medList = []
highList = []
for i in range(0, momMatrix.shape[0]):
    for j in range(1, momMatrix.shape[1]):
        if momMatrix.iloc[i,j] < momPt.iloc[i,0]:
            lowList.append(returnAll.iloc[i,j])
        elif (momPt.iloc[i,0] < momMatrix.iloc[i,j]) and (momMatrix.iloc[i,j] < momPt.iloc[i,1]):
            medList.append(returnAll.iloc[i,j])
        elif momMatrix.iloc[i,j] > momPt.iloc[i,1]:
            highList.append(returnAll.iloc[i,j])
    lowMom.iloc[i,1] = sum(lowList) / len(lowList)
    medMom.iloc[i,1] = sum(medList) / len(medList)
    highMom.iloc[i,1] = sum(highList) / len(highList)

In [129]:
from functools import reduce
momLMH = reduce(lambda  left,right: pd.merge(left,right, on='Date', how='outer'), [lowMom, medMom, highMom])
momLMH.head()

,Date,LowMomentum,MediumMomentum,HighMomentum
0,2021-05-03,0.028317,0.084243,0.167024
1,2021-05-10,-0.051756,-0.020802,0.021574
2,2021-05-17,-0.19436,-0.1585,-0.089848
3,2021-05-24,-0.084516,-0.072168,-0.054362
4,2021-05-31,-0.055801,-0.036919,-0.023456


### Calculate Momentum Factor

In [130]:
momFactor = pd.DataFrame(columns = ['Date', 'MOM_LTW'])
momFactor['Date'] = momMatrix['Date']
momFactor['MOM_LTW'] = momLMH['HighMomentum'] - momLMH['LowMomentum']

In [131]:
momFactor.head(20)

,Date,MOM_LTW
0,2021-05-03,0.138707
1,2021-05-10,0.07333
2,2021-05-17,0.104512
3,2021-05-24,0.030154
4,2021-05-31,0.032345
5,2021-06-07,0.018745
6,2021-06-14,0.018277
7,2021-06-21,0.019533
8,2021-06-28,-0.000888
9,2021-07-05,-0.001338
